<a href="https://colab.research.google.com/github/cerr/pyCERR-notebooks/blob/main/autosegment_CT_Lung_GTV_SMIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The pyCERR SMIT Lung CT GTV Segmentation Model

## Introduction

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the Lung GTV on a lung CT scan using pyCERR.

## Requirements
* Python>=3.8
* Applying this model requires access to a GPU.  

## AI model
* The segmentation model was trained and validated on CT scans used for RT planning. It does not work optimally on diagnostic CTs or scans in positions other than Head First Supine.
* The trained model is distributed along with python libraries and other dependencies via a conda package.

## Required input data
* RT planning CT DICOM

### Running the model

The input DICOM data must be located in inputDicomPath. The directory structure is as follows:

- inputDicomPath
    - Pat ID 1 
    - Pat ID 2
    - Pat ID 3
    - ... 


### License
By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.
    
This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.

You may publish papers and books using results produced using software provided that you reference the appropriate citations:
*  SMIT model: https://arxiv.org/abs/2205.10342
*  CERR library of model implementations: https://doi.org/10.1016/j.ejmp.2020.04.011
*  CERR software: https://doi.org/10.1118/1.1568978
*  CERR radiomics: https://doi.org/10.1002/mp.13046


YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain such licenses to export, re-export, or import as may be required after delivery to you.



## Specify the working directory.
The working directory will host three sub-directories.
- input - This will host sub-directories, one per patient consisting of DICOM files for Lung CT scan.
- output - This will host results of AI segmentation
- session - This will host temporary nii files as I/O from inference wrapper

In [ ]:
import os
workDir = r'/home/uname/SMIT_inference' # r'/content' # for Google Colab
! mkdir -p {workDir}
os.chdir(workDir)

In [ ]:
inputDicomPath = os.path.join(workDir,'input')
os.makedirs(inputDicomPath, exist_ok = True)

outputDicomPath = os.path.join(workDir, 'output')
os.makedirs(outputDicomPath, exist_ok = True)

sessionPath = os.path.join(workDir, 'session')
os.makedirs(sessionPath, exist_ok = True)

## Download planning CT DICOM data for processing

### Option 1: Download data from XNAT source (`getXNATData`) - use only when data is located in an XNAT instance

In [ ]:
!pip install pyxnat

# Function definition: pull data defined in scandict from XNAT
from pyxnat import Interface
import urllib3, shutil
from glob import glob

def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user, verify=False)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['proj']
    subj = scan_entry['subj']
    exp = scan_entry['exp']
    scan_list = scan_entry['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      try:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM').get(downloadDir,extract=True)
      except:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary').get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist

### Option 2:  Download data to `inputDicomPath` (e.g. from an HTTP source) - use when data is hosted in Box

In [ ]:
dataUrl='http://path.to/data'

dataDownloadDir = os.path.join(workDir,'tmp')
os.makedirs(dataDownloadDir, exist_ok=True)
os.chdir(dataDownloadDir)

! wget -O sampleData.gz -L {dataUrl}
! tar xf sampleData.gz -C {inputDicomPath}
! rm sampleData.gz

#unpack and move

In [ ]:
xhost = 'https://your.xnat'
user = 'usr'
scandict = [{'proj':'projectname','subj':'subject_ID','exp':'ID', 'scan_list':['1']}]

dcmdirlist = getXNATData(xhost,user,scandict,inputDicomPath)

### Option 3:  Copy data to `inputDicomPath`

## Install pyCERR for data processing

In [ ]:
#%%capture
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git"

## Change current directory to working dir and define model paths

In [ ]:
os.chdir(workDir)

In [ ]:
wrapperInstallDir = os.path.join(workDir,'CT_Lung_SMIT')
condaEnvDir = os.path.join(wrapperInstallDir,'conda-pack')
condaEnvActivateScript = os.path.join(condaEnvDir, 'bin', 'activate')
wrapperPath = os.path.join(wrapperInstallDir,'bash_run_SMIT_Segmentation.sh')
load_weight_name = os.path.join(wrapperInstallDir,'trained_weights','model.pt')

#pyCERR and filename parameters
scanNum = 0
structName = 'GTV'
scan_basename = 'CT'

In [ ]:
import subprocess
from cerr import plan_container as pc
from cerr.dcm_export import rtstruct_iod
import urllib3, shutil
urllib3.disable_warnings()

## Download the model installer and install the Lung nodule segmentation model

In [ ]:
os.chdir(workDir)
!git clone https://github.com/cerr/model_installer.git

In [ ]:
os.chdir(os.path.join(workDir,'model_installer'))
!./installer.sh -h

In [ ]:
modelOpt = '4' #CT_Lung_SMIT model
pythonOpt = 'C' #download and use pre-packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

## Run the GTV segmentation model, output RTSTRUCT

In [ ]:
for dcmdir in os.scandir(inputDicomPath):

    subj = os.path.basename(dcmdir)
    sessiondir = os.path.join(sessionPath,subj)
    os.makedirs(sessiondir,exist_ok=True)
    
    #Convert input DICOM to NIfTI
    scan_niifile = os.path.join(sessiondir, scan_basename + '.nii')
    planC = pc.loadDcmDir(dcmDir = dcmdir)
    planC.scan[scanNum].saveNii(niiFileName = scan_niifile)
    
    #Run Segmentation
    os.chdir(wrapperInstallDir)
    subprocess.run(f"source " + condaEnvActivateScript + " && source " + wrapperPath + " " + sessiondir + " " + sessiondir + " " + load_weight_name + " " + scan_niifile, shell=True, executable="/bin/bash")
    
    #Import output GTV NIfTI to pyCERR, and generate RTSTRUCT to match parent series
    output_niifile = os.path.join(sessiondir,'model_' + scan_basename + '.nii')
    planC = pc.loadNiiStructure(nii_file_name = output_niifile, assocScanNum = scanNum, planC = planC, labels_dict = {1: structName})
    outputDir = os.path.join(workDir,'output',id)
    os.makedirs(outputDir, exist_ok=True)
    rtstructFile = os.path.join(outputDir, subj+'_RTSTRUCT.dcm')
    structNumV = [0] # Export the first and the last structure in the list
    seriesDescription = structName + "_CT_Lung_GTV_SMIT"
    rtstruct_iod.create(structNumV = structNumV, filePath = rtstructFile, planC = planC, seriesOpts = {'SeriesDescription':seriesDescription})